In [1]:

PKG_path = 'C:/Users/shjo/Desktop/JNUROMS/'
import sys 
sys.path.append(PKG_path)
import Tools.JNUROMS as jr
from Tools.JNU_create import create_bry2, create_bry_VVV
import numpy as np
from netCDF4 import Dataset,MFDataset,date2num,num2date
import os
from scipy.interpolate import griddata
import datetime as dt
from tqdm import tqdm
from copy import deepcopy
My_Bry='J:/tmp/test/Bry_test.nc'
My_Grd='J:/tmp/test/Jelly_dr2021_grd.nc'
OGCM_PATH='D:/SODA/'

Bry_title='Bry_Q1_Rtopo30S_S_soda_50250701_198001_201712'
# My Variables
MyVar={'Layer_N':50,'Vtransform':2,'Vstretching':5,\
       'Theta_s':7,'Theta_b':1,'Tcline':450,'hmin':50}

# OGCM Variables
OGCMVar={'lon_rho':'xt_ocean','lat_rho':'yt_ocean','depth':'st_ocean','time':'time',\
         'lon_u':'xu_ocean','lat_u':'yu_ocean','lon_v':'xu_ocean','lat_v':'yu_ocean',
         'temp':'temp','salt':'salt','u':'u','v':'v','zeta':'ssh'}
conserv=1
OGCMS=[OGCM_PATH+'/'+i for i in os.listdir(OGCM_PATH) if i.endswith('.nc')]

In [2]:
# Get My Grid info
ncG=Dataset(My_Grd)
lonG,latG=ncG['lon_rho'][:],ncG['lat_rho'][:]
angle,topo,mask=ncG['angle'][:],ncG['h'][:],ncG['mask_rho'][:]
ncG.close()

# Get OGCM Grid info
ncO=Dataset(OGCMS[0])
lonO,latO=ncO[OGCMVar['lon_rho']][:],ncO[OGCMVar['lat_rho']][:]
depthO=ncO[OGCMVar['depth']][:]
ncO.close()

In [118]:
lonO.shape

(720,)

In [3]:

t_rng=['1980-01-01 12:00','1980-03-01 12:00']
My_time_ref='days since 1970-1-1 00:00:00'
OGCM_TIMES=MFDataset(OGCM_PATH+'*.nc')[OGCMVar['time']]
TIME_UNIT=OGCM_TIMES.units
OGCM_times=num2date(OGCM_TIMES[:],TIME_UNIT)
Tst=dt.datetime(int(t_rng[0].split('-')[0]), int(t_rng[0].split('-')[1]),int(t_rng[0].split('-')[2][:2]),int(t_rng[0].split('-')[-1].split(' ')[-1][:2]))
Ted=dt.datetime(int(t_rng[1].split('-')[0]), int(t_rng[1].split('-')[1]),int(t_rng[1].split('-')[2][:2]),int(t_rng[1].split('-')[-1].split(' ')[-1][:2]))
TIMES_co=np.where( (OGCM_times>=Tst)&(OGCM_times<=Ted) )[0]
tmp_y,tmp_m,tmp_H=int(t_rng[0].split('-')[0]),int(t_rng[0].split('-')[-1][:2]),int(t_rng[0].split('-')[-1].split(' ')[-1][:2])
tmp_dif=date2num(dt.datetime(tmp_y,tmp_m,tmp_H),TIME_UNIT)-date2num(dt.datetime(tmp_y,tmp_m,tmp_H),My_time_ref)
Bry_time_time=num2date(OGCM_TIMES[TIMES_co]-tmp_dif,My_time_ref)
Bry_time_num=OGCM_TIMES[TIMES_co]-tmp_dif

In [34]:

thO=ncO[OGCMVar['depth']].shape[0]
atG,onG=lonG.shape
cosa_,sina_=np.cos(angle)[-2:],np.sin(angle)[-2:] #NORTHERN BRY
cosa=np.tile( np.tile(cosa_,(thO,1,1)), (len(Bry_time_num),1,1,1) )
sina=np.tile( np.tile(sina_,(thO,1,1)), (len(Bry_time_num),1,1,1) )


In [11]:
NSEW=[True,False,False,True]
create_bry_VVV(My_Bry,mask,topo,MyVar,NSEW,Bry_time_num,My_time_ref,Bry_title,ncFormat='NETCDF4')


!!! Make bry:  north !!!
!!! Make bry:  west !!!


In [193]:
def Calc_bry_co(latG,lonG,latO,lonO,m):
    dlatG,dlatO=np.diff(latG[:,0]),np.diff(latO)
    dlonG,dlonO=np.diff(lonG[0,:]),np.diff(lonO)
    if m==0:
        if np.max(dlatG)>=np.max(dlatO):
            tmp_co=np.min(np.where(latO>=latG[-1,0])[0])
            bry_co=np.array([tmp_co,tmp_co])
        else:
            tmp_co=np.min(np.where(latO>=latG[-1,0])[0])
            bry_co=np.array([tmp_co-1,tmp_co])
    elif m==1:
        if np.max(dlatG)>=np.max(dlatO):
            tmp_co=np.max(np.where(latO<=latG[0,0])[0])
            bry_co=np.array([tmp_co,tmp_co])
        else:
            tmp_co=np.max(np.where(latO<=latG[0,0])[0])
            bry_co=np.array([tmp_co-1,tmp_co])
    elif m==2:
        if np.max(dlatG)>=np.max(dlatO):
            tmp_co=np.max(np.where(lonO<=lonG[-1,0])[0])
            bry_co=np.array([tmp_co,tmp_co])
        else:
            tmp_co=np.max(np.where(lonO<=lonG[-1,0])[0])
            bry_co=np.array([tmp_co,tmp_co-1])
    elif m==3:
        if np.max(dlatG)>=np.max(dlatO):
            tmp_co=np.min(np.where(lonO>=lonG[0,0])[0])
            bry_co=np.array([tmp_co,tmp_co])
        else:
            tmp_co=np.min(np.where(lonO>=lonG[0,0])[0])
            bry_co=np.array([tmp_co,tmp_co-1])
    return bry_co

def rho2u_2d(var):
    N,Lp=var.shape
    L=Lp-1
    var_u=0.5*(var[:,:L]+var[:,1:Lp])
    return var_u
def rho2u_3d(var):
    N,Mp,Lp=var.shape
    L=Lp-1
    var_u=0.5*(var[:,:,:L]+var[:,:,1:Lp])
    return var_u
def rho2u_4d(var):
    T,N,Mp,Lp=var.shape
    L=Lp-1
    var_u=0.5*(var[:,:,:,:L]+var[:,:,:,1:Lp])
    return var_u
def rho2v_3d(var):
    T,Mp,Lp=var.shape
    M=Mp-1
    var_v=0.5*(var[:,:M,:]+var[:,1:Mp,:])
    return var_v
def rho2v_4d(var):
    T,N,Mp,Lp=var.shape
    M=Mp-1
    var_v=0.5*(var[:,:,:M,:]+var[:,:,1:Mp,:])
    return var_v

In [100]:
Calc_bry_co(latG,lonG,latO,lonO,3)

array([253, 252], dtype=int64)

In [101]:
print(Calc_bry_co(latG,lonG,latO,lonO,0))
print(Calc_bry_co(latG,lonG,latO,lonO,1))
print(Calc_bry_co(latG,lonG,latO,lonO,2))
print(Calc_bry_co(latG,lonG,latO,lonO,3))

[219 220]
[216 217]
[252 251]
[253 252]


In [89]:
tmp_co

220

In [74]:
bry_co=np.array([tmp_co,tmp_co])


In [68]:
latG[0,0]

34.09

In [188]:
NSEW=[1,1,1,1]

In [191]:


# Read_Data
OGCM_Data={}#,OGCM_Mask={}
m=-1
for n in NSEW:
    m+=1
    if not NSEW[m]:
        continue  
    
    for i in ['u','v','temp','salt','zeta','ubar','vbar']:
        if m==0:
            mydirc='north'
            bry_co=Calc_bry_co(latG,lonG,latO,lonO,m)
            lonO_s_m,latO_s_m=np.meshgrid(lonO,latO[bry_co])

        elif m==1:
            mydirc='south'
            bry_co=Calc_bry_co(latG,lonG,latO,lonO,m)
            lonO_s_m,latO_s_m=np.meshgrid(lonO,latO[bry_co])

        elif m==2:
            mydirc='east'
            bry_co=Calc_bry_co(latG,lonG,latO,lonO,m)
            lonO_s_m,latO_s_m=np.meshgrid(lonO[bry_co],latO)

        elif m==3:
            mydirc='west'
            bry_co=Calc_bry_co(latG,lonG,latO,lonO,m)
            lonO_s_m,latO_s_m=np.meshgrid(lonO[bry_co],latO)

        print('!!! Data processing : '+i+'_'+mydirc+' !!!')

        if (i=='zeta') or (i=='ubar') or (i=='vbar'):
            if i=='zeta':
                if (m==0) or (m==1):
                    DATA=np.squeeze(MFDataset(OGCMS)[OGCMVar[i]][TIMES_co,bry_co,:])
                    data=np.zeros([len(Bry_time_num),2,lonG.shape[-1]])

                elif (m==2) or (m==3):
                    DATA=np.squeeze(MFDataset(OGCMS)[OGCMVar[i]][TIMES_co,:,bry_co])
                    data=np.zeros([len(Bry_time_num),latG.shape[0],2])

                tmp_mask_=np.invert(DATA.mask)
                
            elif i=='ubar':
                if (m==0) or (m==1):
                    tmp_u=np.squeeze(MFDataset(OGCMS)[OGCMVar['u']][TIMES_co,:,bry_co,:])
                elif (m==2) or (m==3):
                    tmp_u=np.squeeze(MFDataset(OGCMS)[OGCMVar['u']][TIMES_co,:,:,bry_co])
                tmp_mask_=np.invert(tmp_u.mask)[:,:,:,:]
                
                tmp_u[tmp_u.mask]=0
                
                du=np.zeros([tmp_u.shape[0],tmp_u.shape[2],tmp_u.shape[3]])
                zu=np.zeros_like(du)
                dz=np.gradient(-depthO)
                for n in range(len(depthO)):
                    du=du+dz[n]*tmp_u[:,n,:,:].data
                    zu=zu+dz[n]*tmp_mask_[:,n,:,:]
                DATA=du/zu
                # DATA[DATA==0]=np.nan
                tmp_mask_=tmp_mask_[:,0,:,:]
                
            elif i=='vbar':
                if (m==0) or (m==1):
                    tmp_v=np.squeeze(MFDataset(OGCMS)[OGCMVar['v']][TIMES_co,:,bry_co,:])
                elif (m==2) or (m==3):
                    tmp_v=np.squeeze(MFDataset(OGCMS)[OGCMVar['v']][TIMES_co,:,:,bry_co])
                tmp_mask_=np.invert(tmp_v.mask)[:,:,:,:]
                tmp_v[tmp_v.mask]=0
                
                dv=np.zeros([tmp_v.shape[0],tmp_v.shape[2],tmp_v.shape[3]])
                zv=np.zeros_like(dv)
                dz=np.gradient(-depthO)
                for n in range(len(depthO)):
                    dv=dv+dz[n]*tmp_v[:,n,:,:].data
                    zv=zv+dz[n]*tmp_mask_[:,n,:,:]
                DATA=dv/zv
                # DATA[DATA==0]=np.nan
                tmp_mask_=tmp_mask_[:,0,:,:]
                
            for t in range(len(Bry_time_num)):
                tmp_mask=tmp_mask_[t]
                data_=griddata((lonO_s_m[tmp_mask].flatten(),latO_s_m[tmp_mask].flatten()),\
                            DATA[t][tmp_mask].flatten(),(lonO_s_m.flatten(),latO_s_m.flatten()),'nearest')
                data_=data_.reshape(latO_s_m.shape)
        
                # Interp 4 Grid
                if m==0:
                    data_re_=griddata( (lonO_s_m.flatten(),latO_s_m.flatten()), data_.flatten(), (lonG[-2:,:].flatten(),latG[-2:,:].flatten()) ,'cubic' )
                    data[t]=data_re_.reshape(lonG[-2:,:].shape)
                elif m==1:
                    data_re_=griddata( (lonO_s_m.flatten(),latO_s_m.flatten()), data_.flatten(), (lonG[:2,:].flatten(),latG[:2,:].flatten()) ,'cubic' )
                    data[t]=data_re_.reshape(lonG[:2,:].shape)
                elif m==2:
                    data_re_=griddata( (lonO_s_m.flatten(),latO_s_m.flatten()), data_.flatten(), (lonG[:,-2:].flatten(),latG[:,-2:].flatten()) ,'cubic' )
                    data[t]=data_re_.reshape(lonG[:,-2:].shape)
                elif m==3:
                    data_re_=griddata( (lonO_s_m.flatten(),latO_s_m.flatten()), data_.flatten(), (lonG[:,:2].flatten(),latG[:,:2].flatten()) ,'cubic' )
                    data[t]=data_re_.reshape(lonG[:,:2].shape)
            OGCM_Data[i+'_'+mydirc]=data
            
        else:
            
            if (m==0) or (m==1):
                DATA=np.squeeze(MFDataset(OGCMS)[OGCMVar[i]][TIMES_co,:,bry_co,:])
                data=np.zeros([len(Bry_time_num),len(depthO),2,lonG.shape[-1]])

            elif (m==2) or (m==3):
                DATA=np.squeeze(MFDataset(OGCMS)[OGCMVar[i]][TIMES_co,:,:,bry_co])
                data=np.zeros([len(Bry_time_num),len(depthO),latG.shape[0],2])
        
            tmp_mask_=np.invert(DATA.mask)
        
            for t in range(len(Bry_time_num)):
                for d in range(len(depthO)):
                    # Interp mask
                    tmp_mask=tmp_mask_[t,d]
                    data_=griddata((lonO_s_m[tmp_mask].flatten(),latO_s_m[tmp_mask].flatten()),\
                                DATA[t,d][tmp_mask].flatten(),(lonO_s_m.flatten(),latO_s_m.flatten()),'nearest')
                    data_=data_.reshape(latO_s_m.shape)
            
                    # Interp 4 Grid
                if m==0:
                    data_re_=griddata( (lonO_s_m.flatten(),latO_s_m.flatten()), data_.flatten(), (lonG[-2:,:].flatten(),latG[-2:,:].flatten()) ,'cubic' )
                    data[t,d]=data_re_.reshape(lonG[-2:,:].shape)
                    
                elif m==1:
                    data_re_=griddata( (lonO_s_m.flatten(),latO_s_m.flatten()), data_.flatten(), (lonG[:2,:].flatten(),latG[:2,:].flatten()) ,'cubic' )
                    data[t,d]=data_re_.reshape(lonG[:2,:].shape)
                elif m==2:
                    data_re_=griddata( (lonO_s_m.flatten(),latO_s_m.flatten()), data_.flatten(), (lonG[:,-2:].flatten(),latG[:,-2:].flatten()) ,'cubic' )
                    data[t,d]=data_re_.reshape(lonG[:,-2:].shape)
                elif m==3:
                    data_re_=griddata( (lonO_s_m.flatten(),latO_s_m.flatten()), data_.flatten(), (lonG[:,:2].flatten(),latG[:,:2].flatten()) ,'cubic' )
                    data[t,d]=data_re_.reshape(lonG[:,:2].shape)
            OGCM_Data[i+'_'+mydirc]=data



!!! Data processing : u_north !!!
!!! Data processing : v_north !!!
!!! Data processing : temp_north !!!
!!! Data processing : salt_north !!!
!!! Data processing : zeta_north !!!
!!! Data processing : ubar_north !!!
!!! Data processing : vbar_north !!!


C:\Users\shjo\AppData\Local\Temp\ipykernel_6808\1259018329.py:59: RuntimeWarning: invalid value encountered in divide
  DATA=du/zu
C:\Users\shjo\AppData\Local\Temp\ipykernel_6808\1259018329.py:77: RuntimeWarning: invalid value encountered in divide
  DATA=dv/zv


!!! Data processing : u_south !!!
!!! Data processing : v_south !!!
!!! Data processing : temp_south !!!
!!! Data processing : salt_south !!!
!!! Data processing : zeta_south !!!
!!! Data processing : ubar_south !!!
!!! Data processing : vbar_south !!!
!!! Data processing : u_east !!!
!!! Data processing : v_east !!!
!!! Data processing : temp_east !!!
!!! Data processing : salt_east !!!
!!! Data processing : zeta_east !!!
!!! Data processing : ubar_east !!!
!!! Data processing : vbar_east !!!
!!! Data processing : u_west !!!
!!! Data processing : v_west !!!
!!! Data processing : temp_west !!!
!!! Data processing : salt_west !!!
!!! Data processing : zeta_west !!!
!!! Data processing : ubar_west !!!
!!! Data processing : vbar_west !!!


In [194]:
OGCM_Data.keys()

dict_keys(['u_north', 'v_north', 'temp_north', 'salt_north', 'zeta_north', 'ubar_north', 'vbar_north', 'u_south', 'v_south', 'temp_south', 'salt_south', 'zeta_south', 'ubar_south', 'vbar_south', 'u_east', 'v_east', 'temp_east', 'salt_east', 'zeta_east', 'ubar_east', 'vbar_east', 'u_west', 'v_west', 'temp_west', 'salt_west', 'zeta_west', 'ubar_west', 'vbar_west'])

In [195]:

# 4D angle
cosa_,sina_=np.cos(angle)[-2:],np.sin(angle)[-2:] #NORTHERN BRY
cosa=np.tile( np.tile(cosa_,(thO,1,1)), (len(Bry_time_num),1,1,1) )
sina=np.tile( np.tile(sina_,(thO,1,1)), (len(Bry_time_num),1,1,1) )


In [ ]:
#Process 2D vectors
ubar_north= rho2u_2d(OGCM_Data['ubar'][:,-1,:]*cosa[:,0,-1,:]+OGCM_Data['vbar'][:,-1,:]*sina[:,0,-1,:])
vbar_north= rho2v_3d(OGCM_Data['vbar'][:,-2:,:]*cosa[:,0,-2:,:]+OGCM_Data['ubar'][:,-2:,:]*sina[:,0,-2:,:]).squeeze()

#Process 3D vectors
u=rho2u_3d(OGCM_Data['u'][:,:,-1,:]*cosa[:,:,-1,:]+OGCM_Data['v'][:,:,-1,:]*sina[:,:,-1,:])
v=rho2v_4d(OGCM_Data['v'][:,:,-2:,:]*cosa[:,:,-2:,:]-OGCM_Data['u'][:,:,-2:,:]*sina[:,:,-2:,:]).squeeze()